pcfg
VeterbiParser
InsideChartParser

In [2]:
pip install pcfg

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.5.1 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 3.10.1 which is incompatible.



  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 6.8.0
    Uninstalling importlib-metadata-6.8.0:
      Successfully uninstalled importlib-metadata-6.8.0


In [10]:
import nltk
from nltk import PCFG, ViterbiParser, InsideChartParser

In [7]:
grammar = PCFG.fromstring("""
S -> NP VP [1.0]
PP -> P NP [1.0]
VP -> V NP [0.7]
VP -> VP PP [0.3]
NP -> NP PP [0.4]
P -> 'with' [1.0]
V -> 'saw' [1.0]
NP -> 'astronomers' [0.1] | 'ears' [0.18] | 'saw' [0.04] | 'stars' [0.18] | 'telescope' [0.1]
""")

In [15]:
sentence="astronomers saw stars with ears".split()

In [16]:
viterbi_parser = ViterbiParser(grammar)
inside_chart_parser = InsideChartParser(grammar)

In [17]:
for tree in viterbi_parser.parse(sentence):
    print("Viterbi Parse Tree:", tree)

Viterbi Parse Tree: (S
  (NP astronomers)
  (VP (V saw) (NP (NP stars) (PP (P with) (NP ears))))) (p=0.0009072)


In [18]:
for tree in inside_chart_parser.parse(sentence):
    print("InsideChartParser Parse Tree:", tree)

InsideChartParser Parse Tree: (S
  (NP astronomers)
  (VP (V saw) (NP (NP stars) (PP (P with) (NP ears))))) (p=0.0009072)
InsideChartParser Parse Tree: (S
  (NP astronomers)
  (VP (VP (V saw) (NP stars)) (PP (P with) (NP ears)))) (p=0.0006804)


In [ ]:
import pickle

class Rule:
    def __init__(self, left, right, probability):
        self.left = left
        self.right = right
        self.probability = probability

def find_rule_with_word(terminal):
    return [rule for rule in rules if terminal in rule.right]

def find_possible_combinations(x, y):
    return [((x, i), (i+1, y)) for i in range(x, y)]

Z = int(input("Enter no of rules: "))
rules = []
for i in range(Z):
    print("-" * 10)
    print(f"Rule {i+1}")
    key = input("Enter left side of the rule: ").upper()
    value = input("Enter right side of the rule in single string: ").upper().split()
    probability = float(input("Enter probability of the rule: "))
    rule = Rule(key, value, probability)
    rules.append(rule)

pickle.dump(rules, open("rules.data", "wb"))

rules = pickle.load(open("rules.data", "rb"))
words = [input(f"Enter word {i+1}: ").upper() for i in range(int(input("Enter no of words: ")))]

matrix = [[[] for j in range(len(words))] for i in range(len(words))]
for i in range(len(words)):
    for j in range(len(words)):
        if i + j < len(words):
            if i == 0:
                matrix[j][j] = find_rule_with_word(words[j])
            else:
                x = j
                y = i + j
                data = []
                for k in find_possible_combinations(x, y):
                    for left in matrix[k[0][0]][k[0][1]]:
                        for right in matrix[k[1][0]][k[1][1]]:
                            for rule in rules:
                                if rule.right == [left.left, right.left]:
                                    probability = left.probability * right.probability * rule.probability
                                    data.append(Rule(rule.left, rule.right, probability))
                matrix[x][y] = data

if len(matrix[0][-1]) > 0 and all(rule.left == "S" for rule in matrix[0][-1]):
    prob = sum(rule.probability for rule in matrix[0][-1])
    print(f"The probability of the input sentence is {prob}")
else:
    print("The input sentence is not valid according to the grammar rules.")